In [19]:
#imports and setup

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset

import numpy as np
from collections import defaultdict, Counter
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats import entropy

# Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

Looking in indexes: https://download.pytorch.org/whl/cu121


In [20]:
print(torch.cuda.is_available())

True


In [21]:
# Define the transformations (normalization is key for training)
transform = transforms.Compose([
    transforms.ToTensor(),
    # Normalization parameters for CIFAR-10
    # mean for CIFAR-10 = (0.4914, 0.4822, 0.4465)
    #std for CIFAR-10  = (0.2023, 0.1994, 0.2010)
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load original datasets
original_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
original_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [22]:
# Reproducibility
SEED = 32
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ---- AlexNet variant for CIFAR-10 (32x32) ----
class CIFARAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            # 32x32 -> 32x32
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # 32x32 -> 16x16
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 16x16 -> 16x16
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # 16x16 -> 8x8
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 8x8 -> 8x8
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # 8x8 -> 8x8
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # 8x8 -> 8x8
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # 8x8 -> 4x4
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 256 * 4 * 4 = 4096
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256*4*4, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # (N, 256*4*4)
        x = self.classifier(x)
        return x

# Augmentation for training (paper §6.3): flip, ±10° rotation, ±10% translate, ~0.2% zoom
train_transform_w_aug = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=10, translate=(0.10, 0.10), scale=(0.998, 1.002)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# No-aug (for eval and loaders that shouldn't augment)
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [23]:
# Use the already-downloaded datasets but re-wrap them with the correct transforms when needed
full_train_eval = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=eval_transform)
full_test_eval  = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=eval_transform)

NUM_TRAIN = len(full_train_eval)   # 50_000
NUM_TEST  = len(full_test_eval)    # 10_000

#Attacker Dataset - 20K members from CIFAR training, 10K nonmembers from CIFAR training
attacker_idxs = np.random.permutation(np.arange(NUM_TRAIN))

shadow_member_idx = attacker_idxs[:20000]
shadow_nonmembers_idx = attacker_idxs[20000:30000]

Dtrain_attack_members = Subset(full_train_eval, shadow_member_idx.tolist())
Dtrain_attack_nonmembers  = Subset(full_train_eval, shadow_nonmembers_idx.tolist())

# === Summary of Dataset Partitioning (CIFAR-10) ===
print("===== Dataset Partition Summary =====")
print(f"Total CIFAR-10 Train Samples: {len(full_train_eval)}")
print(f"Total CIFAR-10 Test Samples:  {len(full_test_eval)}\n")

print("\n--- Attacker Train Set ---")
print(f"Members (from train):      {len(Dtrain_attack_members)}")
print(f"Non-members:   {len(Dtrain_attack_nonmembers)}")

===== Dataset Partition Summary =====
Total CIFAR-10 Train Samples: 50000
Total CIFAR-10 Test Samples:  10000


--- Attacker Train Set ---
Members (from train):      20000
Non-members:   10000


In [24]:
BATCH_SIZE = 128

def make_subset_dataset(indices, with_aug: bool):
    base = torchvision.datasets.CIFAR10(root='./data', train=True, download=False,
                                        transform=train_transform_w_aug if with_aug else eval_transform)
    return Subset(base, indices.tolist())

train_loader = []
eval_loader  = []

ds_train = make_subset_dataset(shadow_member_idx, with_aug=True)
ds_eval  = make_subset_dataset(shadow_nonmembers_idx, with_aug=False)
train_loader.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True))
eval_loader.append( DataLoader(ds_eval,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True))

# Full test loader for reporting model accuracies (like Table 2)
full_test_loader = DataLoader(full_test_eval, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [25]:
# models = []
# acc_table = []
# for i, tr_loader in enumerate(train_loader, start=1):
#     print(f"\nTraining shadow_model (with augmentation)...")
#     net = build_model()
#     net = train_one_model(net, tr_loader, epochs=60)

#     # Store in-memory
#     models.append(net)

#     # Evaluate accuracy
#     acc = accuracy(net, full_test_loader)
#     acc_table.append((f"f{i}", acc))
#     print(f"Shadow model accuracy on CIFAR-10 test (with aug): {acc:.2f}%")

#      # ---- SAVE TO DISK ----
#     torch.save(net.state_dict(), f"shadow_model.pth")
#     print(f"Saved model as shadow_model.pth")

In [26]:
# ---- weight init helper (optional) ----
def kaiming_init(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

def build_model():
    net = CIFARAlexNet(num_classes=10)
    net.apply(kaiming_init)
    return net

def train_one_model(model, train_loader, epochs=60, device=device):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 45, 55], gamma=0.1)

    model.train()
    for epoch in range(epochs):
        running = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad(set_to_none=True)
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            running += loss.item() * xb.size(0)
        scheduler.step()
        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs} - loss: {running/len(train_loader.dataset):.4f}")
    return model

@torch.no_grad()
def accuracy(model, data_loader, device=device):
    model.eval()
    correct = 0
    total = 0
    for xb, yb in data_loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        pred = torch.argmax(logits, dim=1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
    return 100.0 * correct / total

In [27]:
def load_model(path):
    device = torch.device("cpu")
    model = build_model().to(device)
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    return model

shadow_path = "shadow_model.pth"
shadow_model = load_model(shadow_path)

In [28]:
shadow_eval_members = make_subset_dataset(shadow_member_idx, with_aug=False)
shadow_eval_nonmembers = make_subset_dataset(shadow_nonmembers_idx, with_aug=False)

shadow_eval_members_loader = DataLoader(shadow_eval_members, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
shadow_eval_nonmembers_loader = DataLoader(shadow_eval_nonmembers, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [29]:
#collect features for training shadow model classifier
criterion = nn.CrossEntropyLoss(reduction="none")

def collect_features(model, data_loader, member_label, device = device):
    model.eval()
    all_features = []
    all_membership = []

    for xb, yb in data_loader:
        xb, yb = xb.to(device), yb.to(device)

        logits = model(xb)

        pred = torch.argmax(logits, dim=1)
        correct = (pred == yb).float()

        probs = F.softmax(logits, dim = 1)

        max_prob, _ = probs.max(dim = 1)

        loss = criterion(logits, yb)

        eps = 1e-12
        entropy = -(probs * (probs + eps).log()).sum(dim = 1)

        top_values, _ = probs.topk(2, dim=1)
        margin = top_values[:, 0] - top_values[:, 1]

        model_features = torch.stack([correct, max_prob, loss, entropy, margin], dim = 1)

        all_features.append(model_features.cpu())
        all_membership.append(torch.full((model_features.size(0),), member_label, dtype  = torch.long))

    all_features = torch.cat(all_features, dim = 0).detach().cpu().numpy()
    all_membership = torch.cat(all_membership, dim = 0).detach().cpu().numpy()

    return all_features, all_membership

In [30]:
#collect the shadow model features which will be used to train the attacker
shadow_x_member, shadow_y_member = collect_features(shadow_model, shadow_eval_members_loader, member_label = 1)
shadow_x_nonmember, shadow_y_nonmember = collect_features(shadow_model, shadow_eval_nonmembers_loader, member_label = 0)

#sanity check to make sure there are 20k members with 5 features
print("Shadow Member Feature Shape:", shadow_x_member.shape)
print("Shadow Member Labels Shape:", shadow_y_member.shape)

#sanity check to make sure there are 10k nonmembers with 5 features
print("Shadow Nonmember Feature Shape:", shadow_x_nonmember.shape)
print("Shadow Nonmember Labels Shape:", shadow_y_nonmember.shape)

Shadow Member Feature Shape: (20000, 5)
Shadow Member Labels Shape: (20000,)
Shadow Nonmember Feature Shape: (10000, 5)
Shadow Nonmember Labels Shape: (10000,)


In [31]:
shadow_attack_x = np.vstack([shadow_x_member, shadow_x_nonmember])
shadow_attack_y = np.concatenate([shadow_y_member, shadow_y_nonmember])

In [32]:
shadow_attack_x = np.vstack([shadow_x_member, shadow_x_nonmember])
shadow_attack_y = np.concatenate([shadow_y_member, shadow_y_nonmember])

clf = RandomForestClassifier(n_estimators = 200, max_depth = None, min_samples_split = 2, min_samples_leaf = 1, n_jobs = -1, class_weight = "balanced")
clf.fit(shadow_attack_x, shadow_attack_y)

shadow_attack_scores = clf.predict_proba(shadow_attack_x)[:, 1]

shadow_auc = roc_auc_score(shadow_attack_y, shadow_attack_scores)
print("Shadow AUC:", shadow_auc)

shadow_fpr, shadow_tpr, thresholds = roc_curve(shadow_attack_y, shadow_attack_scores)
shadow_advantage = (shadow_tpr - shadow_fpr).max()

print("Shadow Attack Advantage:", shadow_advantage)

Shadow AUC: 1.0
Shadow Attack Advantage: 1.0


# MCE CODE

In [33]:
# Load the five models
f1 = load_model("f1.pth")
f2 = load_model("f2.pth")
f3 = load_model("f3.pth")
f4 = load_model("f4.pth")
f5 = load_model("f5.pth")

models = [f1, f2, f3, f4, f5]

# (Optional) double-check f1 accuracy achieved in the setup
print("f1 test accuracy:", accuracy(f1, full_test_loader))


f1 test accuracy: 73.59


In [34]:
#collect features from any of the models to use for attack
def collect_features(dataset, models, member_label, attack_type, ece_scores, device = device):
    all_features = []
    all_membership = []
    eps = 1e-12

    for i in range(len(dataset)):
        if i  % 1000 == 0:
            print(i)
        xb, yb = dataset[i]
        xb = xb.to(device)

        if torch.is_tensor(yb):
            yb = int(yb.item())

        #call specific prediction function for each attack type
        if attack_type == 'CDE':
            probs, excluded_idx = confidence_deviation_predict(xb, models, device)
        elif attack_type == 'HCE':
            probs, excluded_idx = calibration_weighted_predict(xb, models, device)
        elif attack_type == 'KLD':
            probs, excluded_idx = hybrid_kl_predict(xb, models, device)
        else:
            probs, excluded_idx = mce_oracle_predict(xb, models, device)
        max_prob = float(probs.max())

        pred = int(np.argmax(probs))
        correct = 1.0 if pred == yb else 0.0

        true_class = float(probs[yb])
        loss = -np.log(true_class + eps)

        entropy = -float(np.sum(probs * np.log(probs + eps)))

        sorted_probs = np.sort(probs)[::-1]
        margin = float(sorted_probs[0] - sorted_probs[1])

        all_features.append([correct, max_prob, loss, entropy, margin])
        all_membership.append(member_label)

    all_features = np.array(all_features, dtype=np.float32)
    all_membership = np.array(all_membership, dtype=np.int64)

    return all_features, all_membership

In [35]:
from collections import Counter
import numpy as np
import torch


def mce_oracle_predict(x_tensor, ensemble_models, device):
    all_predictions = []
    for model in ensemble_models:
        model.eval() # Set model to evaluation mode
        with torch.no_grad():
            output = model(x_tensor.unsqueeze(0).to(device))
            # Get probabilities
            probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]
            all_predictions.append(probabilities)

    predictions = np.array(all_predictions) # Shape: (num_models, num_classes)

    top_labels = [np.argmax(p) for p in predictions]

    # Handle cases where top_labels might be empty
    if not top_labels:
        return np.zeros(predictions.shape[1]), 0

    # Find the majority label among the top predictions of all models
    majority_label_counts = Counter(top_labels)
    majority_label = majority_label_counts.most_common(1)[0][0]

    # Calculate confidence of each model in the majority label
    label_confidences = [p[majority_label] for p in predictions]
    # Exclude the model with the *lowest* confidence in the majority class
    excluded_idx = np.argmax(label_confidences)

    # Remove the predictions of the excluded model and average the rest
    remaining_preds = np.delete(predictions, excluded_idx, axis=0)
    final_pred = np.mean(remaining_preds, axis=0)

    return final_pred, excluded_idx

def evaluate_mce_on_test(dataset, models, device):
    correct = 0
    exclusion_counts = np.zeros(len(models), dtype=int)
    total_samples = len(dataset)

    print(f"\nEvaluating MCE Oracle on {total_samples} test samples...")
    for i in range(total_samples):
        x_tensor, y_true = dataset[i]

        pred_probs, excluded = mce_oracle_predict(x_tensor, models, device)
        y_pred = np.argmax(pred_probs)

        if y_pred == y_true:
            correct += 1

        exclusion_counts[excluded] += 1

    accuracy = correct / total_samples
    print(f"\nMCE Oracle Accuracy on full test set: {accuracy * 100:.2f}%")
    print("Model Exclusion Counts:")
    for i, count in enumerate(exclusion_counts):
        print(f"   • Model {i+1}: excluded {count} times")

    return accuracy, exclusion_counts


# acc_mce, exclusion_stats = evaluate_mce_on_test(full_test_eval, models, device)

# MCE ATTACK

In [47]:
#collect MCE features which will be used with classifer
mce_x_member, mce_y_member = collect_features(full_train_eval, models, member_label = 1, attack_type = 'MCE', ece_scores = None, device = device)
mce_x_nonmember, mce_y_nonmember = collect_features(full_test_eval, models, member_label = 0, attack_type = 'MCE', ece_scores = None, device = device)

print("MCE Features:")
print("Member Feature Shape:", mce_x_member.shape)
print("Member Labels Shape:", mce_y_member.shape)
print("Nonmember Feature Shape:", mce_x_nonmember.shape)
print("Nonmember Labels Shape:", mce_y_nonmember.shape)

mce_attack_x = np.vstack([mce_x_member, mce_x_nonmember])
mce_attack_y = np.concatenate([mce_y_member, mce_y_nonmember])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
MCE Features:
Member Feature Shape: (50000, 5)
Member Labels Shape: (50000,)
Nonmember Feature Shape: (10000, 5)
Nonmember Labels Shape: (10000,)


In [51]:
attack_aucs = []

In [52]:
mce_attack_scores = clf.predict_proba(mce_attack_x)[:, 1]

mce_auc = roc_auc_score(mce_attack_y, mce_attack_scores)
print("MCE AUC:", mce_auc)

mce_fpr, mce_tpr, thresholds = roc_curve(mce_attack_y, mce_attack_scores)
mce_advantage = (mce_tpr - mce_fpr).max()

attack_aucs.append({'model': 'MCE', 'AUC': mce_auc, 'Advantage': mce_advantage})
print(attack_aucs)

MCE AUC: 0.510471232
[{'model': 'MCE', 'AUC': np.float64(0.510471232), 'Advantage': np.float64(0.01758000000000004)}]


In [53]:
for attack in attack_aucs:
    print(f"Model under attack: {attack['model']}")
    print(f"Attack AUC: {attack['AUC']}")
    print(f"Attack Advantage: {attack['Advantage']}")

Model under attack: MCE
Attack AUC: 0.510471232
Attack Advantage: 0.01758000000000004
